<a href="https://colab.research.google.com/github/baroodb/code/blob/main/videos2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import pickle

In [2]:
!git clone https://github.com/jurjsorinliviu/Sports-Type-Classifier.git clone

Cloning into 'clone'...
remote: Enumerating objects: 14521, done.
remote: Total 14521 (delta 0), reused 0 (delta 0), pack-reused 14521
Receiving objects: 100% (14521/14521), 592.88 MiB | 26.97 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Checking out files: 100% (14619/14619), done.


In [3]:
%%writefile train.py
import argparse
from imutils import paths
import os
import cv2
import numpy as np

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import AveragePooling2D, Flatten, Dense, Dropout, Input


LABELS = ["weight_lifting", "tennis", "football"]


parser = argparse.ArgumentParser()
parser.add_argument('-d', '--dataset', help='This is the path to the dataset', required=True)
parser.add_argument('-m', '--model', help='Path to save the model to', required=True)
parser.add_argument('-e', '--epochs', type=int,  help='The number of epochs', required=True)
parser.add_argument('-l', '--label-bin', help='Path to save the label binarizer to...')
parser.add_argument('-p', '--plot', help='Path to output the plot..')

args = vars(parser.parse_args())

list_files = list(paths.list_images(args['dataset']))

images = []
labels = []

for path in list_files:
  label = path.split(os.path.sep)[-2]


  if label not in LABELS:
    continue
  
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (224, 224))

  images.append(img)
  labels.append(label)

images = np.array(images)
labels = np.array(labels)


print(images.shape)
print(labels.shape)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print('INFO: Printing some elements from the label binarizer')
print(labels)

# split the data
train_ds, val_ds, train_labels, val_labels = train_test_split(images, 
                                                              labels, 
                                                              test_size=0.2, 
                                                              stratify=labels, 
                                                              random_state=42)
assert train_ds.shape[0] == train_labels.shape[0]
assert val_ds.shape[0] == val_labels.shape[0]

# Create the data augmentation 
train_augmentation = tf.keras.preprocessing.image.ImageDataGenerator(
    	rotation_range=30,
      zoom_range=0.15,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.15,
      horizontal_flip=True,
      fill_mode="nearest"
)

val_augmentation = tf.keras.preprocessing.image.ImageDataGenerator()

mean = np.array([123.68, 116.779, 103.939], dtype="float32")

train_augmentation.mean = mean
val_augmentation.mean = mean


#builing the model

base = ResNet50(include_top=False)
for layer in base.layers:
  layer.trainable = False

inputs = Input(shape=(224, 224, 3))
x = base(inputs, training=False)
x = AveragePooling2D((7, 7))(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(len(LABELS), activation='softmax')(x)

model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

print('INFO: Printing the model summary')
print(model.summary())

print('INFO: Compiling the model')
model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-4, momentum=0.9, decay=1e-4/args['epochs']), 
              metrics=['accuracy'])

#
print('INFO: Launching the training job')

H = model.fit(x=train_augmentation.flow(train_ds, train_labels, batch_size=32), 
              steps_per_epoch=len(train_ds) // 32, 
              validation_data=val_augmentation.flow(val_ds, val_labels), 
              validation_steps=len(val_ds) // 32, 
              epochs=args['epochs'])


Writing train.py


In [4]:
!python train.py --dataset /content/clone/data --epochs 10 --model /model

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
(2070, 224, 224, 3)
(2070,)
INFO: Printing some elements from the label binarizer
[[0 1 0]
 [0 1 0]
 [0 1 0]
 ...
 [1 0 0]
 [1 0 0]
 [1 0 0]]
2021-09-15 23:38:51.920782: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-15 23:38:52.411317: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-15 23:38:52.412238: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there mus